<div style="display: flex; align-items: center; gap: 40px;">

<img src="https://play-lh.googleusercontent.com/_O9p4Z4yucA2NLmZBu9mTJCuBwXeT9NcbtrDN6I8gKlkIPRySV0adOmbyipjSj9Gew" width="130">



<div>
  <h2>SUTRA by TWO Platforms </h2>
  <p>SUTRA is a family of large multi-lingual language (LMLMs) models pioneered by Two Platforms. SUTRA’s dual-transformer approach extends the power of both MoE and Dense AI language model architectures, delivering cost-efficient multilingual capabilities for over 50+ languages. It powers scalable AI applications for conversation, search, and advanced reasoning, ensuring high-performance across diverse languages, domains and applications.</p>

  <h2>RAG Fusion Using SUTRA</h2>
  <p>RAG-Fusion is an enhanced version of the traditional Retrieval-Augmented Generation (RAG) model. In RAG-Fusion, after receiving a query, the model first generates related sub-queries using a large language model. These sub-queries help find more relevant documents. Instead of simply sending the retrieved documents to the model, RAG-Fusion uses a technique called Reciprocal Rank Fusion (RRF) to score and reorder the documents based on their relevance. The best-ranked documents are then used to generate a more accurate response.</p>
</div>


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1mzQCIF3qHedBsfg9CWAyBgvywj_H1HHX?usp=sharing)

## Get Your API Keys

Before you begin, make sure you have:

1. A SUTRA API key (Get yours at [TWO AI's SUTRA API page](https://www.two.ai/sutra/api))
2. Basic familiarity with Python and Jupyter notebooks

This notebook is designed to run in Google Colab, so no local Python installation is required.

###🔧 1. Install Required Libraries

In [ ]:
!pip install -qU langchain_openai langchain_community chromadb langsmith

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 82.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.1/361.1 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 78.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 114.6 MB/s eta 

###🔑 2. Set Environment Variables (API Keys)

In [ ]:
import os
from google.colab import userdata

# Set the API key from Colab secrets
os.environ["SUTRA_API_KEY"] = userdata.get("SUTRA_API_KEY")
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

###🔹 Step 3: Load Documents and Split

In [ ]:
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = CSVLoader("./context.csv")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
documents = text_splitter.split_documents(documents)

###🔹 Step 4: Setup Embeddings and Vector Store (ChromaDB)

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma

embeddings = OpenAIEmbeddings()

vectorstore = Chroma.from_documents(documents, embeddings)

###🔹 Step 5: Create Retriever

In [ ]:
retriever = vectorstore.as_retriever()

###🔹 Step 6: Initialize Sutra LLM and Langsmith Prompt Client

In [ ]:
from langchain_openai import ChatOpenAI
from langsmith import Client
import os

client = Client()

llm = ChatOpenAI(
    api_key=os.getenv("SUTRA_API_KEY"),
    base_url="https://api.two.ai/v2",
    model="sutra-v2"
)


/usr/local/lib/python3.11/dist-packages/langsmith/client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


###🔹 Step 7: Load RAG-Fusion Query Generation Prompt from Langsmith

In [ ]:
from langchain_core.output_parsers import StrOutputParser

prompt = client.pull_prompt("langchain-ai/rag-fusion-query-generation")

generate_queries = (
    prompt
    | ChatOpenAI(temperature=0, api_key=os.getenv("SUTRA_API_KEY"), base_url="https://api.two.ai/v2", model="sutra-v2")
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)


###🔹 Step 8: Reciprocal Rank Fusion (RRF) Function

In [ ]:
from langchain.load import dumps, loads

def reciprocal_rank_fusion(results: list[list], k=60):
    fused_scores = {}
    for docs in results:
        # docs sorted by relevance descending
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            fused_scores[doc_str] += 1 / (rank + k)
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    return reranked_results

###🔹 Step 9: Build RAG-Fusion Chain

In [ ]:
chain = generate_queries | retriever.map() | reciprocal_rank_fusion

###🔹 Step 10: Test the Fusion Chain

In [ ]:
query = "what are points on a mortgage"
results = chain.invoke(query)
print("RRF Results Top 3 Documents:")
for doc, score in results[:3]:
    print(f"Score: {score:.3f} - Doc excerpt: {doc.page_content[:200]}...\n")

RRF Results Top 3 Documents:
Score: 0.049 - Doc excerpt: context: ["Discount points, also called mortgage points or simply points, are a form of pre-paid interest available in the United States when arranging a mortgage. One point equals one percent of the ...

Score: 0.033 - Doc excerpt: up-front payment. For each point purchased, the loan rate is typically reduced by anywhere from 1/8% (0.125%) to 1/4% (0.25%).Selling the property or refinancing prior to this break-even point will re...

Score: 0.033 - Doc excerpt: rate.Points may also be purchased to reduce the monthly payment for the purpose of qualifying for a loan.  Loan qualification based on monthly income versus the monthly loan payment may sometimes only...



###🔹 Step 11: Final RAG Answer Chain


In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context.
If you don't find the answer in the context, just say that you don't know.

Context: {context}

Question: {question}
"""

prompt_rag = ChatPromptTemplate.from_template(template)

rag_fusion_chain = (
    {
        "context": chain,
        "question": RunnablePassthrough()
    }
    | prompt_rag
    | llm
    | StrOutputParser()
)

final_answer = rag_fusion_chain.invoke(query)
print("Final RAG-Fusion Answer:\n", final_answer)

Final RAG-Fusion Answer:
 Points on a mortgage, also known as discount points or mortgage points, are a form of pre-paid interest that borrowers can pay to reduce the interest rate on their loan. One point equals one percent of the loan amount. By paying points, borrowers can lower their monthly payments and the overall cost of the loan. Points can also be used to qualify for a loan by reducing the monthly payment required based on income.


###🔹 Step 12: Prepare Data for Evaluation

In [ ]:
questions = ["what are points on a mortgage"]
responses = []
contexts = []
ground_truths = ["Points, sometimes also called a 'discount point', are a form of pre-paid interest."]

for q in questions:
    responses.append(rag_fusion_chain.invoke(q))
    contexts.append([doc.page_content for doc in retriever.invoke(q)])

data = {
    "query": questions,
    "response": responses,
    "context": contexts,
    "ground_truth": ground_truths
}


<ipython-input-12-eeecdd16137b>:8: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  contexts.append([doc.page_content for doc in retriever.get_relevant_documents(q)])


###🔹 Step 13: Create Dataset and DataFrame

In [ ]:
from datasets import Dataset
import pandas as pd

dataset = Dataset.from_dict(data)
df = pd.DataFrame(dataset)
df

,query,response,context,ground_truth
0,what are points on a mortgage,"Points on a mortgage, also known as discount p...","[context: [""Discount points, also called mortg...","Points, sometimes also called a 'discount poin..."
